In [22]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
import pycountry
import openpyxl
import random

random.seed(42)

ocde = [
    'Australia',
    'Austria',
    'Belgium',
    'Canada',
    'Chile',
    'Colombia',
    'Costa Rica',
    'Czechia',
    'Denmark',
    'Estonia',
    'Finland',
    'France',
    'Germany',
    'Greece',
    'Hungary',
    'Iceland',
    'Ireland',
    'Israel',
    'Italy',
    'Japan',
    'South Korea',
    'Latvia',
    'Lithuania',
    'Luxembourg',
    'Mexico',
    'Netherlands',
    'New Zealand',
    'Norway',
    'Poland',
    'Portugal',
    'Slovakia',
    'Slovenia',
    'Spain',
    'Sweden',
    'Switzerland',
    'Türkiye',
    'United Kingdom',
    'United States'
]

df = pd.read_json('../../data/tw/prokanon.tw_combs4.json')
df.loc[df['count_max'] <= 1000, 'count_max'] = df['count_min']

df['count_max'] = df['count_max'].apply(lambda x: x if x >= 1062 else random.randint(1, 1062))

df = df.loc[~(df['age_range'] == 'AGE_18_TO_24')]
df = df.loc[~(df['age_range'] == 'AGE_18_TO_34')]
df = df.loc[~(df['age_range'] == 'AGE_21_TO_34')]
df = df.loc[~(df['age_range'] == 'AGE_35_TO_49')]

df.sort_values(by='count_max',inplace=True)


df['cumsum'] = df['count_max'].cumsum()

suma = df['count_max'].sum()

df['cdf'] = df['cumsum'] / suma

df

,_id,country,platform,age_range,gender,count_max,count_min,cumsum,cdf
9839,{'$oid': '65b7cbaa60aac84946a82f1b'},GQ,4,AGE_25_TO_49,FEMALE,1,0,1,1.641113e-09
8567,{'$oid': '65b7934c60aac84946a7c1ac'},PN,4,AGE_OVER_50,FEMALE,1,0,2,3.282226e-09
10242,{'$oid': '65ba09a760c90558368dee2b'},MG,3,AGE_OVER_50,MALE,1,0,3,4.923339e-09
7225,{'$oid': '65b774bf60aac84946a7574c'},MS,1,AGE_13_TO_24,MALE,2,0,5,8.205566e-09
7837,{'$oid': '65b7761960aac84946a76746'},FM,1,AGE_OVER_50,FEMALE,2,0,7,1.148779e-08
...,...,...,...,...,...,...,...,...,...
148,{'$oid': '65b0e65c17f3c67cf28664c5'},US,0,AGE_25_TO_49,FEMALE,11462880,10371177,545384867,8.950383e-01
112,{'$oid': '65b0e64817f3c67cf2866409'},US,0,AGE_13_TO_24,MALE,13389263,12114095,558774130,9.170116e-01
3980,{'$oid': '65b76d7f60aac84946a70326'},JP,0,AGE_13_TO_24,FEMALE,13479745,12195960,572253875,9.391334e-01
4012,{'$oid': '65b76d9260aac84946a703f5'},JP,0,AGE_25_TO_49,FEMALE,15391739,13925859,587645614,9.643929e-01


In [23]:
df[df['count_max']<=1000][['count_max','count_min']].value_counts()

count_max  count_min
848        0            9
323        0            9
821        0            8
687        0            8
854        0            7
                       ..
502        0            1
810        0            1
105        0            1
501        0            1
368        0            1
Name: count, Length: 918, dtype: int64

In [24]:
df[df['count_min']==961]

,_id,country,platform,age_range,gender,count_max,count_min,cumsum,cdf
6159,{'$oid': '65b7726160aac84946a73bcc'},CR,4,AGE_OVER_50,FEMALE,1062,961,1344655,0.002207
3583,{'$oid': '65b76c9a60aac84946a6f900'},GE,4,AGE_OVER_50,FEMALE,1062,961,1345717,0.002208


In [25]:
df['count_max'].value_counts().sort_index()

count_max
1           3
2           3
3           3
4           3
6           4
           ..
11462880    1
13389263    1
13479745    1
15391739    1
21696894    1
Name: count, Length: 2963, dtype: int64

In [26]:
len(df[df['count_max'] == 0])/len(df)

0.0

In [27]:
df_aux = df['count_max'].value_counts().sort_index()[0:973]
df_aux = df_aux * df_aux.index
df_aux.sum()

np.int64(1345717)

In [28]:
df['count_max'].sum()

np.int64(609342508)

In [29]:
df_aux.sum()/df['count_max'].sum()

np.float64(0.0022084738588432763)

In [30]:
df_aux = df['count_max'].value_counts().sort_index()[0:1]
df_aux = df_aux * df_aux.index
df_aux.sum()

np.int64(3)

In [31]:
df_aux.sum()/df['count_max'].sum()

np.float64(4.923339436545596e-09)

In [32]:
df['count_max'].value_counts().sort_index()

count_max
1           3
2           3
3           3
4           3
6           4
           ..
11462880    1
13389263    1
13479745    1
15391739    1
21696894    1
Name: count, Length: 2963, dtype: int64

In [33]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = (res_geq090['cdf'] - 0.1).abs().idxmin()

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = (res_geq095['cdf'] - 0.05).abs().idxmin()

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = (res_geq099['cdf'] - 0.01).abs().idxmin()

results_99 = res_geq099.loc[idx_min]

In [34]:
results = pd.DataFrame(columns=['K^4_90', 'K^4_95', 'K^4_99', 'RK^4_90', 'RK^4_95', 'RK^4_99'])
results['K^4_90'] = results_90
results['K^4_95'] = results_95
results['K^4_99'] = results_99

In [35]:
results = results.T[['count_max']].T.reset_index(drop = True)
results['RK^4_90'] = (results['K^4_90'] / suma) * 100
results['RK^4_95'] = (results['K^4_95'] / suma) * 100
results['RK^4_99'] = (results['K^4_99'] / suma) * 100

In [36]:
results.to_parquet('../../data/analyzed/tw/TW_K^4_X_general.parquet')

In [37]:
latex_table=results.to_latex(index=False,float_format="%.2e",escape=False)
print(latex_table)

\begin{tabular}{llllll}
\toprule
K^4_90 & K^4_95 & K^4_99 & RK^4_90 & RK^4_95 & RK^4_99 \\
\midrule
182195 & 74041 & 11722 & 2.99e-02 & 1.22e-02 & 1.92e-03 \\
\bottomrule
\end{tabular}



In [38]:
countries_ocde_a2 = pd.DataFrame(pd.DataFrame(ocde,columns=['country'])['country'].apply(lambda x: pycountry.countries.lookup(x).alpha_2))

In [39]:
df_ocde = df[df['country'].apply(lambda x: pycountry.countries.lookup(x).alpha_2).isin(countries_ocde_a2['country'])].copy()
df_ocde.sort_values(by='count_max',inplace=True)

df_ocde['cumsum'] = df_ocde['count_max'].cumsum()

suma = df_ocde['count_max'].sum()

df_ocde['cdf'] = df_ocde['cumsum'] / suma

In [40]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = (res_geq090['cdf'] - 0.1).abs().idxmin()

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = (res_geq095['cdf'] - 0.05).abs().idxmin()

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = (res_geq099['cdf'] - 0.01).abs().idxmin()

results_99 = res_geq099.loc[idx_min]

results = pd.DataFrame(columns=['K^4_90', 'K^4_95', 'K^4_99', 'RK^4_90', 'RK^4_95', 'RK^4_99'])
results['K^4_90'] = results_90
results['K^4_95'] = results_95
results['K^4_99'] = results_99

results = results.T[['count_max']].T.reset_index(drop = True)
results['RK^4_90'] = (results['K^4_90'] / suma) * 100
results['RK^4_95'] = (results['K^4_95'] / suma) * 100
results['RK^4_99'] = (results['K^4_99'] / suma) * 100

results.to_parquet('../../data/analyzed/tw/TW_K^4_X_general_ocde.parquet')
results

,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
0,182195,74041,11722,0.048392,0.019666,0.003113


In [41]:
df_ocde[df_ocde['count_max']==0]['country'].drop_duplicates().count()

np.int64(0)

In [42]:
len(ocde)

38